# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Итог по первому пункту
Данные прочитаны, все в норме, пропусков нет

## Обучение и проверка модели

Мы собираемся для каждого региона посмтроить **линейную модель регрессии** для предсказания его объемов.  Для этого мы в функции реализуем разбивку и построение модели, а на выход поставим целевой выпуск, предсказанные выпуски и rmse.

In [8]:
def make_pred(data):
    features = data.drop(['product', 'id'], axis=1)
    target = data['product'].reset_index(drop=True)
    features_train,features_valid, target_train, target_valid = train_test_split(features,
                                                                                        target,
                                                                                        test_size=0.25,
                                                                                        random_state=8634)
    print('features_train shape:',features_train.shape)
    print('features_valid shape:', features_valid.shape)
    print('target_train shape:', target_train.shape)
    print('target_valid shape:', target_valid.shape)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    pred = model.predict(features_valid)
    rmse = math.sqrt(mse(target_valid, pred))
    mean = pd.Series(pred).mean()
    print('RMSE for data is:', rmse, '\nMean product for region:', pd.Series(pred).mean())
    return target_valid, pred, rmse

In [9]:
target_valid_1, pred_1, rmse_1 = make_pred(data_1)

features_train shape: (75000, 3)
features_valid shape: (25000, 3)
target_train shape: (75000,)
target_valid shape: (25000,)
RMSE for data is: 37.76165036875815 
Mean product for region: 92.28589465885058


In [10]:
target_valid_2, pred_2, rmse_2 = make_pred(data_2)

features_train shape: (75000, 3)
features_valid shape: (25000, 3)
target_train shape: (75000,)
target_valid shape: (25000,)
RMSE for data is: 0.887954623256421 
Mean product for region: 68.889547953874


In [11]:
target_valid_3, pred_3, rmse_3 = make_pred(data_3)

features_train shape: (75000, 3)
features_valid shape: (25000, 3)
target_train shape: (75000,)
target_valid shape: (25000,)
RMSE for data is: 39.94921336783633 
Mean product for region: 95.40159787949625


### Вывод по пункту:
Модели обучены, нужные данные сохранены, идем дальше. Из вышеперечисленных данные видно, что во втором регионе у нас хоть и самый низкие средний продукт, но и RMSE у нас там в разы меньше. Регион 1 и 3 плюс минус идентичны по характеристикам и имеют средний продукт в районе 94, в то время как второй регион - в районе 70.

## Подготовка к расчёту прибыли

In [12]:
ONE_PROD_PRICE = 450000
BUDGET = 10000000000
enough_product = BUDGET / ONE_PROD_PRICE
print('Минимальный объем сырья для безубыточности:', round(enough_product), 'barrels')

Минимальный объем сырья для безубыточности: 22222 barrels


<div class="alert alert-block alert-info">
<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами.
</div>

In [13]:
data = [data_1, data_2, data_3]
for i in [1,2,3]:
    print(f'Средний запас в регионе, данные которые лежат в {i}-м датасете,', end=' ')
    print(f'равен {round(200 * data[i-1]["product"].mean())}') # так как скважин - 200 шт

Средний запас в регионе, данные которые лежат в 1-м датасете, равен 18500
Средний запас в регионе, данные которые лежат в 2-м датасете, равен 13765
Средний запас в регионе, данные которые лежат в 3-м датасете, равен 19000


### Вывод по пункту:
Мы сохранили нужные данные в константах, посчитали, какой объем нужен для безубыточности региона и поняли, что средние показатели нас не устраивают (надо выбирать лучшие). Иначе - убыток

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена корректно, сравнение првоедено, с выводом согласен.
</div>

## Расчёт прибыли и рисков 

Напишем функцию, которая рассчитывает наибольшую прибыль по предсказанным данным

In [14]:
def profit_func(pred, target, count):
    pred = pd.Series(pred, index=target.index)

    pred = pred.sort_values(ascending=False)
    target = target[pred.index][:count]

    all_barr = target.sum()
    profit = all_barr * ONE_PROD_PRICE - BUDGET
    return profit

Реализуем метод Bootstrap, логика которой такова: выбираем случайные 500 точек, для них по предсказанным значениям берем 200 лучших и считаем прибыль. Таких прибылей у нас 1000. И на этих данных мы можем усреднить для региона среднюю прибыль, указать интервалы для прибыли и риск убыточности, который равен кол-ву отрицательных прибылей деленное на всего таких прибылей

In [15]:
def make_proba(target, predict):
    state = np.random.RandomState(132)
    profit = list()
    c = 0
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predict_subsample = predict[target_subsample.index]
        profit_unit = profit_func(predict_subsample,target_subsample, 200)
        profit.append(profit_unit)
        if profit_unit < 0:
            c +=1
            
    profit = pd.Series(profit)
    mean_profit = profit.mean()
    lower_bord = profit.quantile(0.025)
    upper_bord = profit.quantile(0.975)
    risk_prob = c / len(profit)

    return mean_profit, lower_bord, upper_bord, risk_prob

**Считаем** теперь для каждого региона вышеперечисленные характеристики

In [16]:
targets = (target_valid_1,target_valid_2,target_valid_3)
preds = (pred_1,pred_2,pred_3)

def make_answers(t_arr, p_arr):
    for i in range(3):
        mean, l, u, r = make_proba(t_arr[i], p_arr[i])
        print(f'Средняя прибыль в регионе {i+1} равна:', round(mean, 2))
        print(f'Границы 95% доверит интервала: from {l:.2f} to {u:.2f}')
        print(f'Риск убытков равен: {r:.2%}\n')

In [17]:
make_answers(targets, preds)

Средняя прибыль в регионе 1 равна: 451940895.69
Границы 95% доверит интервала: from -72630144.90 to 941528405.39
Риск убытков равен: 4.10%

Средняя прибыль в регионе 2 равна: 519572780.97
Границы 95% доверит интервала: from 81002192.83 to 941945848.36
Риск убытков равен: 0.90%

Средняя прибыль в регионе 3 равна: 417043975.68
Границы 95% доверит интервала: from -125519692.83 to 957438893.37
Риск убытков равен: 6.70%



# Итоговый ответ:
Очевидно, что нужно выбрать **второй регион**, так как: 

-Там самая высокая средняя прибыль

-В доверительных интервалах нет убытков

-Самый низкий риск получить убыток

Также стоит отметить, что для второго региона наша модель дает наименьший RMSE, то есть точность прогноза максимальна из всех

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован